In [ ]:
import os
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-03 11:37:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-12-03 11:37:11 (8.04 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True)# Read in the data from an S3 Bucket
auto_df = df.dropna(how='any')
auto_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [ ]:
auto_df.count()

3514045

In [ ]:
from pyspark.sql.functions import to_date
import pandas as pd
from pyspark.sql.types import IntegerType
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = auto_df[["review_id", "customer_id", "product_parent", "review_date"]]
review_id_df = auto_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))
review_id_df = review_id_df.withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))
review_id_df.show()
review_id_df.dtypes

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [18]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = auto_df[["product_id", "product_title"]]
products_df = products_df.drop_duplicates()
products_df.show()
products_df.dtypes


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002OKKCO8|06-09 KAWASAKI ZX...|
|B00OZXP3BY|Oem Factory Stock...|
|B00HA26HZS|Suzuki Hayabusa G...|
|B00BITHKD2|PT Auto Warehouse...|
|B000688VNE|Hi-Lift Jack Hi-L...|
|B004GCP4GS|OE Replacement Au...|
|B0006HPA02|MSD 32829 8.5mm S...|
|B00EK6NS22|uxcell® Car Bike ...|
|B00WMR1I64|MICTUNING mt00880...|
|B009S31VVW|Dorman 81702 Ford...|
|B00QJ8FA70|Parts Player New ...|
|B00HZOCCBY|MAXFLOORMAT Floor...|
|B00DZPC5NM|Dodge Grand Carav...|
|B001VKT9IW|HushMat 10301 Ult...|
|B009SEJI66|2011-2013 Toyota ...|
|B007QBKODO|ACDelco 18A2729 P...|
|B007LIBJPO|Custom Accessorie...|
|B009YKQR2C|Zone Tech 4pcs 20...|
|B00062YAZ8|Edelbrock 4201 El...|
|B008IAHBWY|Jeep Wrangler Tj ...|
+----------+--------------------+
only showing top 20 rows



[('product_id', 'string'), ('product_title', 'string')]

In [ ]:
customers_df = auto_df.groupBy("customer_id").count()
customers_df = customers_df.withColumn("customer_id",customers_df["customer_id"].cast(IntegerType()))
customers_df = customers_df.withColumn("count",customers_df["count"].cast(IntegerType()))
customers_df = customers_df.withColumnRenamed("count", "customer_count") 
customers_df.show()
customers_df.dtypes

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    2583392|             4|
|     129005|             1|
|    3962209|             2|
|   16092109|             1|
|   46841294|             1|
|   42126527|             4|
|   47138302|             1|
|   10484128|             2|
|    5427648|             1|
|     580770|             1|
|   14286306|             6|
|   13880860|             1|
|   24095000|             2|
|   50189080|             1|
|   50583551|             4|
|   15562874|             1|
|   51068983|             3|
|   10245571|             6|
|   28932014|             8|
|   42891605|             4|
+-----------+--------------+
only showing top 20 rows



[('customer_id', 'int'), ('customer_count', 'int')]

In [ ]:
vine_df = auto_df[["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]]
vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))
vine_df = vine_df.withColumn("helpful_votes",vine_df["helpful_votes"].cast(IntegerType()))
vine_df = vine_df.withColumn("total_votes",vine_df["total_votes"].cast(IntegerType()))
vine_df.show()
vine_df.dtypes

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>"
config = {"user":"root", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='auto_review', mode=mode, properties=config)

Py4JJavaError: ignored

In [19]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='auto_products', mode=mode, properties=config)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [20]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='auto_customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='auto_vine', mode=mode, properties=config)